In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', None)  # Display full content of each column
pd.set_option('display.max_columns', None)   # Display all columns
pd.set_option('display.width', 5000)         # Set display width

plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Frequency
--

In [ ]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from datetime import datetime

# ------------------------
# Data Loading and Preprocessing
# ------------------------
df = pd.read_csv("Merge_Proccessed.csv")  # Replace with your actual data source

# Convert Transaction_Date to datetime and extract components for breakdown analysis
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
df['Year'] = df['Transaction_Date'].dt.year
df['Month_Num'] = df['Transaction_Date'].dt.month
df['Month'] = df['Transaction_Date'].dt.strftime('%B')  # e.g., January, February, etc.
df['Week_Num'] = df['Transaction_Date'].dt.strftime('%U').astype(int) + 1  # Week number starting at 1
df['Day'] = df['Transaction_Date'].dt.date

# ------------------------
# Helper Functions for Breakdown Analysis
# ------------------------
def generate_summary(df_filtered, group_col):
    """
    Group the filtered DataFrame by a column and summarize Debit, Credit, and Balance.
    Also count the total transactions.
    """
    df_grouped = df_filtered.groupby(group_col).agg({
        'Debit': 'sum', 
        'Credit': 'sum', 
        'Balance': 'last'
    }).reset_index()
    
    transaction_counts = df_filtered.groupby(group_col).size().reset_index(name='Total_Transactions')
    df_summary = df_grouped.merge(transaction_counts, on=group_col)
    # Recalculate balance as income minus spending
    df_summary['Balance'] = df_summary['Credit'] - df_summary['Debit']
    return df_summary

def create_bar_chart(df_summary, x_col, title):
    """
    Create a grouped bar chart comparing Debit and Credit.
    """
    fig = px.bar(
        df_summary, 
        x=x_col, 
        y=['Debit', 'Credit'], 
        text_auto='.2s',
        labels={'value': 'Amount', 'variable': 'Type'}, 
        barmode='group',
        color_discrete_map={'Debit': 'lightcoral', 'Credit': 'lightblue'},
        hover_data=['Total_Transactions', 'Balance']
    )
    fig.update_layout(
        title=title, 
        xaxis_title=x_col, 
        yaxis_title='Amount', 
        plot_bgcolor='whitesmoke'
    )
    return fig

# ------------------------
# Helper Function for Summary Boxes with Custom Styles
# ------------------------
def summary_box(title, value, bg_color, text_color):
    return html.Div([
        html.H4(title, style={'fontFamily': 'Arial', 'color': text_color}),
        html.P(value, style={'fontSize': '28px', 'fontWeight': 'bold', 'color': text_color, 'margin': '0'})
    ], style={
        'padding': '20px',
        'margin': '5px',
        'backgroundColor': bg_color,
        'borderRadius': '10px',
        'textAlign': 'center',
        'flex': '1',
        'boxShadow': '2px 2px 2px lightgrey'
    })

# ------------------------
# Initialize Dash App with Bootstrap for Modals
# ------------------------
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# ------------------------
# App Layout
# ------------------------
app.layout = html.Div([
    html.H1("Comprehensive Transaction Analysis Dashboard", style={'textAlign': 'center'}),
    dcc.Tabs([
        # ----- Tab 1: General Analysis -----
        dcc.Tab(label="General Analysis", children=[
            html.Div([
                html.H2("General Transaction Analysis"),
                html.Label("Select Date Range"),
                dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date=df['Transaction_Date'].min().date(),
                    end_date=df['Transaction_Date'].max().date(),
                    display_format='YYYY-MM-DD'
                ),
                # Summary Boxes container
                html.Div(id='general-summary-boxes', 
                         style={'display': 'flex', 'justifyContent': 'space-around', 'padding': '10px'}),
                # Grid layout: 2 charts per row (4 charts total)
                html.Div([
                    # Frequency Distribution Chart Container
                    html.Div([
                        dcc.Graph(id='freq-bar', config={'displayModeBar': True}, style={'height': '300px'}),
                        html.Button("Expand", id="btn-expand-freq", n_clicks=0, 
                                    style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                    ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                        'position': 'relative', 'margin': '10px'}),
                    # Top Transactions Chart Container
                    html.Div([
                        dcc.Graph(id='top-transactions', config={'displayModeBar': True}, style={'height': '300px'}),
                        html.Button("Expand", id="btn-expand-top", n_clicks=0, 
                                    style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                    ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                        'position': 'relative', 'margin': '10px'}),
                    # Amount Distribution Chart Container
                    html.Div([
                        dcc.Graph(id='amount-pie', config={'displayModeBar': True}, style={'height': '300px'}),
                        html.Button("Expand", id="btn-expand-amount", n_clicks=0, 
                                    style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                    ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                        'position': 'relative', 'margin': '10px'}),
                    # Date Trend Chart Container
                    html.Div([
                        dcc.Graph(id='date-trend', config={'displayModeBar': True}, style={'height': '300px'}),
                        html.Button("Expand", id="btn-expand-trend", n_clicks=0, 
                                    style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                    ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                        'position': 'relative', 'margin': '10px'}),
                ], style={'textAlign': 'center'})
            ], style={'padding': '10px'}),
            # Hidden Stores to hold figures for modals (General Analysis)
            dcc.Store(id='store-freq-bar'),
            dcc.Store(id='store-top-transactions'),
            dcc.Store(id='store-amount-pie'),
            dcc.Store(id='store-date-trend'),
            # Modals for Expanded Views (General Analysis)
            dbc.Modal([
                dbc.ModalHeader("Frequency Distribution"),
                dbc.ModalBody(dcc.Graph(id='modal-freq-bar', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-freq", className="ml-auto"))
            ], id="modal-freq", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Top Transactions"),
                dbc.ModalBody(dcc.Graph(id='modal-top-transactions', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-top", className="ml-auto"))
            ], id="modal-top", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Transaction Amount Distribution"),
                dbc.ModalBody(dcc.Graph(id='modal-amount-pie', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-amount", className="ml-auto"))
            ], id="modal-amount", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Transaction Trends Over Time"),
                dbc.ModalBody(dcc.Graph(id='modal-date-trend', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-trend", className="ml-auto"))
            ], id="modal-trend", size="xl", is_open=False)
        ]),
        
        # ----- Tab 2: Breakdown Analysis -----
        dcc.Tab(label="Breakdown Analysis", children=[
            html.Div([
                html.H2("Transaction Breakdown Analysis"),
                html.Div([
                    html.Label("Select Year"),
                    dcc.Dropdown(
                        id='year-dropdown',
                        options=[{'label': str(y), 'value': y} for y in sorted(df['Year'].unique())],
                        placeholder='Select Year'
                    )
                ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
                html.Div([
                    html.Label("Select Month"),
                    dcc.Dropdown(id='month-dropdown', placeholder='Select Month')
                ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
                html.Div([
                    html.Label("Select Week"),
                    dcc.Dropdown(id='week-dropdown', placeholder='Select Week')
                ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'margin': '10px'}),
            ], style={'padding': '10px'}),
            # Grid layout for Breakdown Analysis Charts (4 charts arranged in 2 rows)
            html.Div([
                html.Div([
                    dcc.Graph(id='yearly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
                    html.Button("Expand", id="btn-expand-yearly", n_clicks=0,
                                style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                    'position': 'relative', 'margin': '10px'}),
                html.Div([
                    dcc.Graph(id='monthly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
                    html.Button("Expand", id="btn-expand-monthly", n_clicks=0,
                                style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                    'position': 'relative', 'margin': '10px'}),
            ], style={'textAlign': 'center'}),
            html.Div([
                html.Div([
                    dcc.Graph(id='weekly-summary', config={'displayModeBar': True}, style={'height': '300px'}),
                    html.Button("Expand", id="btn-expand-weekly", n_clicks=0,
                                style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                    'position': 'relative', 'margin': '10px'}),
                html.Div([
                    dcc.Graph(id='daily-summary', config={'displayModeBar': True}, style={'height': '300px'}),
                    html.Button("Expand", id="btn-expand-daily", n_clicks=0,
                                style={'position': 'absolute', 'bottom': '10px', 'right': '10px'})
                ], className="graph-card", style={'width': '45%', 'display': 'inline-block', 
                                                    'position': 'relative', 'margin': '10px'}),
            ], style={'textAlign': 'center'}),
            # Hidden Stores for Breakdown Analysis modals
            dcc.Store(id='store-yearly-summary'),
            dcc.Store(id='store-monthly-summary'),
            dcc.Store(id='store-weekly-summary'),
            dcc.Store(id='store-daily-summary'),
            # Modals for Expanded Breakdown Analysis Charts
            dbc.Modal([
                dbc.ModalHeader("Yearly Summary"),
                dbc.ModalBody(dcc.Graph(id='modal-yearly-summary', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-yearly", className="ml-auto"))
            ], id="modal-yearly", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Monthly Summary"),
                dbc.ModalBody(dcc.Graph(id='modal-monthly-summary', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-monthly", className="ml-auto"))
            ], id="modal-monthly", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Weekly Summary"),
                dbc.ModalBody(dcc.Graph(id='modal-weekly-summary', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-weekly", className="ml-auto"))
            ], id="modal-weekly", size="xl", is_open=False),
            dbc.Modal([
                dbc.ModalHeader("Daily Summary"),
                dbc.ModalBody(dcc.Graph(id='modal-daily-summary', style={'height': '600px'})),
                dbc.ModalFooter(dbc.Button("Close", id="close-modal-daily", className="ml-auto"))
            ], id="modal-daily", size="xl", is_open=False),
        ])
    ])
])

# ------------------------
# Callback for General Analysis Summary Boxes
# ------------------------
@app.callback(
    Output('general-summary-boxes', 'children'),
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_general_summary_boxes(start_date, end_date):
    df_filtered = df[(df['Transaction_Date'] >= start_date) & (df['Transaction_Date'] <= end_date)]
    income = df_filtered['Credit'].sum()
    expenditure = df_filtered['Debit'].sum()
    balance = income - expenditure
    total_tx = len(df_filtered)
    boxes = [
         summary_box("Income", f"${income:,.2f}", bg_color='#d4edda', text_color='#155724'),
         summary_box("Expenditure", f"${expenditure:,.2f}", bg_color='#f8d7da', text_color='#721c24'),
         summary_box("Balance", f"${balance:,.2f}", bg_color='#d1ecf1', text_color='#0c5460'),
         summary_box("Total Transactions", total_tx, bg_color='#fff3cd', text_color='#856404')
    ]
    return boxes

# ------------------------
# Callbacks for General Analysis Tab
# ------------------------
@app.callback(
    [Output('freq-bar', 'figure'),
     Output('top-transactions', 'figure'),
     Output('amount-pie', 'figure'),
     Output('date-trend', 'figure'),
     Output('store-freq-bar', 'data'),
     Output('store-top-transactions', 'data'),
     Output('store-amount-pie', 'data'),
     Output('store-date-trend', 'data')],
    [Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_general_analysis(start_date, end_date):
    # Filter data based on selected date range
    df_filtered = df[(df['Transaction_Date'] >= start_date) & (df['Transaction_Date'] <= end_date)].copy()
    # Create Transaction_Key: Use Recipient_Name if available, otherwise use Transaction_ID
    df_filtered['Transaction_Key'] = df_filtered.get('Recipient_Name', df_filtered['Transaction_ID'])
    
    # Frequency Distribution
    transaction_counts = df_filtered['Transaction_Key'].value_counts()
    frequency_groups = {
        '1 Transaction': sum(transaction_counts == 1),
        '1-5 Transactions': sum((transaction_counts > 1) & (transaction_counts <= 5)),
        '5-10 Transactions': sum((transaction_counts > 5) & (transaction_counts <= 10)),
        '10-35 Transactions': sum((transaction_counts > 10) & (transaction_counts <= 35)),
        '35-50 Transactions': sum((transaction_counts > 35) & (transaction_counts <= 50)),
        'More than 50': sum(transaction_counts > 50)
    }
    freq_fig = px.bar(
        x=list(frequency_groups.keys()),
        y=list(frequency_groups.values()),
        labels={'x': "Transaction Frequency", 'y': "Count"},
        title="Transaction Frequency Distribution"
    )
    
    # Top 10 Frequent Transactions
    top_transactions = transaction_counts.head(10)
    top_fig = px.bar(
        x=top_transactions.values,
        y=top_transactions.index,
        orientation='h',
        labels={'x': "Transaction Count", 'y': "Transaction Key"},
        title="Top 10 Frequent Transactions"
    )
    
    # Transaction Amount Distribution (based on Debit amounts)
    transaction_ranges = {
        '0-50': sum(df_filtered['Debit'] <= 50),
        '50-100': sum((df_filtered['Debit'] > 50) & (df_filtered['Debit'] <= 100)),
        '100-500': sum((df_filtered['Debit'] > 100) & (df_filtered['Debit'] <= 500)),
        '500-1000': sum((df_filtered['Debit'] > 500) & (df_filtered['Debit'] <= 1000)),
        '1000-5000': sum((df_filtered['Debit'] > 1000) & (df_filtered['Debit'] <= 5000)),
        '5000-10000': sum((df_filtered['Debit'] > 5000) & (df_filtered['Debit'] <= 10000)),
        '10000+': sum(df_filtered['Debit'] > 10000)
    }
    amount_pie = px.pie(
        names=list(transaction_ranges.keys()),
        values=list(transaction_ranges.values()),
        title="Transaction Amount Distribution"
    )
    
    # Date-wise Transaction Trends (summing Debit amounts over time)
    date_trend_df = df_filtered.groupby('Transaction_Date')['Debit'].sum().reset_index()
    trend_fig = px.line(
        date_trend_df,
        x="Transaction_Date",
        y="Debit",
        title="Transaction Trends Over Time",
        labels={'Transaction_Date': "Date", 'Debit': "Total Amount"}
    )
    
    return freq_fig, top_fig, amount_pie, trend_fig, freq_fig, top_fig, amount_pie, trend_fig

# ------------------------
# Callbacks for Breakdown Analysis Tab
# ------------------------
# Update Month dropdown based on selected Year
@app.callback(
    Output('month-dropdown', 'options'),
    Input('year-dropdown', 'value')
)
def update_month_dropdown(selected_year):
    if not selected_year:
        return []
    months = df[df['Year'] == selected_year][['Month_Num', 'Month']].drop_duplicates()
    return [{'label': row['Month'], 'value': row['Month_Num']} for _, row in months.iterrows()]

# Update Week dropdown based on selected Year and Month
@app.callback(
    Output('week-dropdown', 'options'),
    [Input('year-dropdown', 'value'),
     Input('month-dropdown', 'value')]
)
def update_week_dropdown(selected_year, selected_month):
    if not selected_year or not selected_month:
        return []
    weeks = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)][['Week_Num']].drop_duplicates()
    return [{'label': f'Week {w}', 'value': w} for w in sorted(weeks['Week_Num'])]

# Yearly Summary: Group by Month within the selected Year
@app.callback(
    [Output('yearly-summary', 'figure'),
     Output('store-yearly-summary', 'data')],
    Input('year-dropdown', 'value')
)
def update_yearly_summary(selected_year):
    if not selected_year:
        return {}, {}
    df_filtered = df[df['Year'] == selected_year]
    df_summary = generate_summary(df_filtered, 'Month')
    fig = create_bar_chart(df_summary, 'Month', f'Year {selected_year} Summary')
    return fig, fig

# Monthly Summary: Group by Week within the selected Year and Month
@app.callback(
    [Output('monthly-summary', 'figure'),
     Output('store-monthly-summary', 'data')],
    [Input('year-dropdown', 'value'),
     Input('month-dropdown', 'value')]
)
def update_monthly_summary(selected_year, selected_month):
    if not selected_year or not selected_month:
        return {}, {}
    df_filtered = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)]
    df_summary = generate_summary(df_filtered, 'Week_Num')
    df_summary['Week_Num'] = df_summary['Week_Num'].apply(lambda x: f'Week {x}')
    fig = create_bar_chart(df_summary, 'Week_Num', f'{df_filtered["Month"].iloc[0]} Summary')
    return fig, fig

# Weekly Summary: Group by Day within the selected Year, Month, and Week
@app.callback(
    [Output('weekly-summary', 'figure'),
     Output('store-weekly-summary', 'data')],
    [Input('year-dropdown', 'value'),
     Input('month-dropdown', 'value'),
     Input('week-dropdown', 'value')]
)
def update_weekly_summary(selected_year, selected_month, selected_week):
    if not selected_year or not selected_month or not selected_week:
        return {}, {}
    df_filtered = df[
        (df['Year'] == selected_year) &
        (df['Month_Num'] == selected_month) &
        (df['Week_Num'] == selected_week)
    ]
    df_summary = generate_summary(df_filtered, 'Day')
    fig = create_bar_chart(df_summary, 'Day', f'Week {selected_week} Summary')
    return fig, fig

# Daily Summary: Group by Day for detailed info in the selected Week
@app.callback(
    [Output('daily-summary', 'figure'),
     Output('store-daily-summary', 'data')],
    [Input('year-dropdown', 'value'),
     Input('month-dropdown', 'value'),
     Input('week-dropdown', 'value')]
)
def update_daily_summary(selected_year, selected_month, selected_week):
    if not selected_year or not selected_month or not selected_week:
        return {}, {}
    df_filtered = df[
        (df['Year'] == selected_year) &
        (df['Month_Num'] == selected_month) &
        (df['Week_Num'] == selected_week)
    ]
    df_summary = generate_summary(df_filtered, 'Day')
    fig = create_bar_chart(df_summary, 'Day', f'Daily Summary for Week {selected_week}')
    return fig, fig

# ------------------------
# Callbacks to Toggle General Analysis Modals
# ------------------------
@app.callback(
    Output("modal-freq", "is_open"),
    [Input("btn-expand-freq", "n_clicks"), Input("close-modal-freq", "n_clicks")],
    [State("modal-freq", "is_open")]
)
def toggle_modal_freq(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-top", "is_open"),
    [Input("btn-expand-top", "n_clicks"), Input("close-modal-top", "n_clicks")],
    [State("modal-top", "is_open")]
)
def toggle_modal_top(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-amount", "is_open"),
    [Input("btn-expand-amount", "n_clicks"), Input("close-modal-amount", "n_clicks")],
    [State("modal-amount", "is_open")]
)
def toggle_modal_amount(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-trend", "is_open"),
    [Input("btn-expand-trend", "n_clicks"), Input("close-modal-trend", "n_clicks")],
    [State("modal-trend", "is_open")]
)
def toggle_modal_trend(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

# ------------------------
# Callbacks to Toggle Breakdown Analysis Modals
# ------------------------
@app.callback(
    Output("modal-yearly", "is_open"),
    [Input("btn-expand-yearly", "n_clicks"), Input("close-modal-yearly", "n_clicks")],
    [State("modal-yearly", "is_open")]
)
def toggle_modal_yearly(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-monthly", "is_open"),
    [Input("btn-expand-monthly", "n_clicks"), Input("close-modal-monthly", "n_clicks")],
    [State("modal-monthly", "is_open")]
)
def toggle_modal_monthly(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-weekly", "is_open"),
    [Input("btn-expand-weekly", "n_clicks"), Input("close-modal-weekly", "n_clicks")],
    [State("modal-weekly", "is_open")]
)
def toggle_modal_weekly(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

@app.callback(
    Output("modal-daily", "is_open"),
    [Input("btn-expand-daily", "n_clicks"), Input("close-modal-daily", "n_clicks")],
    [State("modal-daily", "is_open")]
)
def toggle_modal_daily(expand, close, is_open):
    if expand or close:
        return not is_open
    return is_open

# ------------------------
# Callbacks to Update Modal Graphs (General Analysis)
# ------------------------
@app.callback(
    Output("modal-freq-bar", "figure"),
    Input("store-freq-bar", "data")
)
def update_modal_freq_bar(store_data):
    return store_data

@app.callback(
    Output("modal-top-transactions", "figure"),
    Input("store-top-transactions", "data")
)
def update_modal_top(store_data):
    return store_data

@app.callback(
    Output("modal-amount-pie", "figure"),
    Input("store-amount-pie", "data")
)
def update_modal_amount(store_data):
    return store_data

@app.callback(
    Output("modal-date-trend", "figure"),
    Input("store-date-trend", "data")
)
def update_modal_trend(store_data):
    return store_data

# ------------------------
# Callbacks to Update Modal Graphs (Breakdown Analysis)
# ------------------------
@app.callback(
    Output("modal-yearly-summary", "figure"),
    Input("store-yearly-summary", "data")
)
def update_modal_yearly_summary(store_data):
    return store_data

@app.callback(
    Output("modal-monthly-summary", "figure"),
    Input("store-monthly-summary", "data")
)
def update_modal_monthly_summary(store_data):
    return store_data

@app.callback(
    Output("modal-weekly-summary", "figure"),
    Input("store-weekly-summary", "data")
)
def update_modal_weekly_summary(store_data):
    return store_data

@app.callback(
    Output("modal-daily-summary", "figure"),
    Input("store-daily-summary", "data")
)
def update_modal_daily_summary(store_data):
    return store_data

# ------------------------
# Run the Dash App
# ------------------------
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)


Category
--

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load your categorized transactions dataset
df = pd.read_csv("CSVS/Categorized_Transactions.csv")

df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce')  # Ensure proper datetime format
df['Month'] = df['Transaction_Date'].dt.strftime('%B')  # Extract Month as string

# Ensure Amount Spent is calculated
df['Amount_Spent'] = df['Debit'].fillna(0)  # Using Debit column to represent spending

# Group data for visualization
category_summary = df.groupby("Transaction_Category")["Amount_Spent"].sum().reset_index()
subcategory_summary = df.groupby(["Transaction_Category", "Sub_Category"])["Amount_Spent"].sum().reset_index()
monthly_summary = df.groupby("Month")["Amount_Spent"].sum().reset_index()

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Expense Dashboard"

# Layout
app.layout = html.Div([
    html.H1("💰 Expense Dashboard", style={'textAlign': 'center', 'color': '#4A90E2'}),
    html.P("Track where your money is going!", style={'textAlign': 'center'}),
    
    dcc.Graph(id='category-pie-chart'),
    dcc.Graph(id='subcategory-bar-chart'),
    dcc.Graph(id='monthly-trend-chart')
])

# Callbacks for interactivity
@app.callback(
    Output('category-pie-chart', 'figure'),
    Output('subcategory-bar-chart', 'figure'),
    Output('monthly-trend-chart', 'figure'),
    Input('category-pie-chart', 'clickData')
)
def update_charts(clickData):
    # Pie Chart (Category-wise Spending)
    fig1 = px.pie(category_summary, values='Amount_Spent', names='Transaction_Category', 
                  title="Category-wise Spending Breakdown",
                  color_discrete_sequence=px.colors.qualitative.Pastel)
    
    # Bar Chart (Subcategory-wise Spending)
    fig2 = px.bar(subcategory_summary, x='Sub_Category', y='Amount_Spent', 
                  title="Subcategory-wise Breakdown", color='Transaction_Category',
                  color_discrete_sequence=px.colors.qualitative.Safe)
    
    # Line Chart (Monthly Trend)
    fig3 = px.line(monthly_summary, x='Month', y='Amount_Spent', markers=True, 
                   title="Monthly Spending Trend", line_shape='spline',
                   color_discrete_sequence=['#2E91E5'])
    
    return fig1, fig2, fig3

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)


In [5]:
# import pandas as pd
# import plotly.express as px
# import dash
# from dash import dcc, html, Input, Output
# from datetime import datetime

# # ------------------------
# # Data Loading and Preprocessing
# # ------------------------
# df = pd.read_csv("Merge_Proccessed.csv")  # Replace with your actual data source

# # Convert Transaction_Date to datetime and extract components for breakdown analysis
# df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])
# df['Year'] = df['Transaction_Date'].dt.year
# df['Month_Num'] = df['Transaction_Date'].dt.month
# df['Month'] = df['Transaction_Date'].dt.strftime('%B')  # e.g., January, February, etc.
# df['Week_Num'] = df['Transaction_Date'].dt.strftime('%U').astype(int) + 1  # Week number starting at 1
# df['Day'] = df['Transaction_Date'].dt.date

# # ------------------------
# # Helper Functions for Breakdown Analysis
# # ------------------------
# def generate_summary(df_filtered, group_col):
#     """
#     Group the filtered DataFrame by a column and summarize Debit, Credit, and Balance.
#     Also count the total transactions.
#     """
#     df_grouped = df_filtered.groupby(group_col).agg({
#         'Debit': 'sum', 
#         'Credit': 'sum', 
#         'Balance': 'last'
#     }).reset_index()

#     transaction_counts = df_filtered.groupby(group_col).size().reset_index(name='Total_Transactions')
#     df_summary = df_grouped.merge(transaction_counts, on=group_col)
#     # Recalculate balance as income minus spending
#     df_summary['Balance'] = df_summary['Credit'] - df_summary['Debit']
#     return df_summary

# def create_bar_chart(df_summary, x_col, title):
#     """
#     Create a grouped bar chart comparing Debit and Credit.
#     """
#     fig = px.bar(
#         df_summary, 
#         x=x_col, 
#         y=['Debit', 'Credit'], 
#         text_auto='.2s',
#         labels={'value': 'Amount', 'variable': 'Type'}, 
#         barmode='group',
#         color_discrete_map={'Debit': 'lightcoral', 'Credit': 'lightblue'},
#         hover_data=['Total_Transactions', 'Balance']
#     )
#     fig.update_layout(
#         title=title, 
#         xaxis_title=x_col, 
#         yaxis_title='Amount', 
#         plot_bgcolor='whitesmoke'
#     )
#     return fig

# # ------------------------
# # Helper function for Summary Boxes with custom styles
# # ------------------------
# def summary_box(title, value, bg_color, text_color):
#     return html.Div([
#         html.H4(title, style={'fontFamily': 'Arial', 'color': text_color}),
#         html.P(value, style={'fontSize': '28px', 'fontWeight': 'bold', 'color': text_color, 'margin': '0'})
#     ], style={
#         'padding': '20px',
#         'margin': '5px',
#         'backgroundColor': bg_color,
#         'borderRadius': '10px',
#         'textAlign': 'center',
#         'flex': '1',
#         'boxShadow': '2px 2px 2px lightgrey'
#     })

# # ------------------------
# # Initialize Dash App and Layout with Tabs
# # ------------------------
# app = dash.Dash(__name__)

# app.layout = html.Div([
#     html.H1("Comprehensive Transaction Analysis Dashboard", style={'textAlign': 'center'}),
#     dcc.Tabs([
#         # ----- Tab 1: General Analysis -----
#         dcc.Tab(label="General Analysis", children=[
#             # Summary Boxes for General Analysis (updates based on Date Range)
#             html.Div(id='general-summary-boxes', 
#                      style={'display': 'flex', 'justify-content': 'space-around', 'padding': '10px'}),
#             html.Div([
#                 html.H2("General Transaction Analysis"),
#                 html.Label("Select Date Range"),
#                 dcc.DatePickerRange(
#                     id='date-picker-range',
#                     start_date=df['Transaction_Date'].min().date(),
#                     end_date=df['Transaction_Date'].max().date(),
#                     display_format='YYYY-MM-DD'
#                 ),
#                 dcc.Graph(id='freq-bar'),
#                 dcc.Graph(id='top-transactions'),
#                 dcc.Graph(id='amount-pie'),
#                 dcc.Graph(id='date-trend')
#             ], style={'padding': 10})
#         ]),
#         # ----- Tab 2: Breakdown Analysis -----
#         dcc.Tab(label="Breakdown Analysis", children=[
#             # Summary Boxes for Breakdown Analysis (updates based on Year/Month/Week selection)
#             html.Div(id='breakdown-summary-boxes', 
#                      style={'display': 'flex', 'justify-content': 'space-around', 'padding': '10px'}),
#             html.Div([
#                 html.H2("Transaction Breakdown Analysis"),
#                 html.Div([
#                     html.Label("Select Year"),
#                     dcc.Dropdown(
#                         id='year-dropdown',
#                         options=[{'label': str(y), 'value': y} for y in sorted(df['Year'].unique())],
#                         placeholder='Select Year'
#                     )
#                 ], style={'width': '24%', 'display': 'inline-block', 'verticalAlign': 'top'}),
#                 html.Div([
#                     html.Label("Select Month"),
#                     dcc.Dropdown(id='month-dropdown', placeholder='Select Month')
#                 ], style={'width': '24%', 'display': 'inline-block', 'marginLeft': '1%', 'verticalAlign': 'top'}),
#                 html.Div([
#                     html.Label("Select Week"),
#                     dcc.Dropdown(id='week-dropdown', placeholder='Select Week')
#                 ], style={'width': '24%', 'display': 'inline-block', 'marginLeft': '1%', 'verticalAlign': 'top'}),
#             ], style={'padding': 10}),
#             html.Div([
#                 dcc.Graph(id='yearly-summary')
#             ]),
#             html.Div([
#                 dcc.Graph(id='monthly-summary')
#             ]),
#             html.Div([
#                 dcc.Graph(id='weekly-summary')
#             ]),
#             html.Div([
#                 dcc.Graph(id='daily-summary')
#             ])
#         ])
#     ])
# ])

# # ------------------------
# # Callbacks for General Analysis Tab Graphs
# # ------------------------
# @app.callback(
#     [Output('freq-bar', 'figure'),
#      Output('top-transactions', 'figure'),
#      Output('amount-pie', 'figure'),
#      Output('date-trend', 'figure')],
#     [Input('date-picker-range', 'start_date'),
#      Input('date-picker-range', 'end_date')]
# )
# def update_general_analysis(start_date, end_date):
#     # Filter data based on selected date range
#     df_filtered = df[(df['Transaction_Date'] >= start_date) & (df['Transaction_Date'] <= end_date)].copy()

#     # Create Transaction_Key: Use Recipient_Name if available, otherwise use Transaction_ID
#     df_filtered['Transaction_Key'] = df_filtered['Recipient_Name'].fillna(df_filtered['Transaction_ID'])
    
#     # Frequency Distribution
#     transaction_counts = df_filtered['Transaction_Key'].value_counts()
#     frequency_groups = {
#         '1 Transaction': sum(transaction_counts == 1),
#         '1-5 Transactions': sum((transaction_counts > 1) & (transaction_counts <= 5)),
#         '5-10 Transactions': sum((transaction_counts > 5) & (transaction_counts <= 10)),
#         '10-35 Transactions': sum((transaction_counts > 10) & (transaction_counts <= 35)),
#         '35-50 Transactions': sum((transaction_counts > 35) & (transaction_counts <= 50)),
#         'More than 50': sum(transaction_counts > 50)
#     }
#     freq_fig = px.bar(
#         x=list(frequency_groups.keys()), 
#         y=list(frequency_groups.values()),
#         labels={'x': "Transaction Frequency", 'y': "Count"},
#         title="Transaction Frequency Distribution"
#     )
    
#     # Top 10 Frequent Transactions
#     top_transactions = transaction_counts.head(10)
#     top_fig = px.bar(
#         x=top_transactions.values, 
#         y=top_transactions.index, 
#         orientation='h',
#         labels={'x': "Transaction Count", 'y': "Transaction Key"},
#         title="Top 10 Frequent Transactions"
#     )
    
#     # Transaction Amount Distribution (based on Debit amounts)
#     transaction_ranges = {
#         '0-50': sum(df_filtered['Debit'] <= 50),
#         '50-100': sum((df_filtered['Debit'] > 50) & (df_filtered['Debit'] <= 100)),
#         '100-500': sum((df_filtered['Debit'] > 100) & (df_filtered['Debit'] <= 500)),
#         '500-1000': sum((df_filtered['Debit'] > 500) & (df_filtered['Debit'] <= 1000)),
#         '1000-5000': sum((df_filtered['Debit'] > 1000) & (df_filtered['Debit'] <= 5000)),
#         '5000-10000': sum((df_filtered['Debit'] > 5000) & (df_filtered['Debit'] <= 10000)),
#         '10000+': sum(df_filtered['Debit'] > 10000)
#     }
#     amount_pie = px.pie(
#         names=list(transaction_ranges.keys()), 
#         values=list(transaction_ranges.values()),
#         title="Transaction Amount Distribution"
#     )
    
#     # Date-wise Transaction Trends (summing Debit amounts over time)
#     date_trend = df_filtered.groupby('Transaction_Date')['Debit'].sum().reset_index()
#     trend_fig = px.line(
#         date_trend, 
#         x="Transaction_Date", 
#         y="Debit",
#         title="Transaction Trends Over Time",
#         labels={'Transaction_Date': "Date", 'Debit': "Total Amount"}
#     )
    
#     return freq_fig, top_fig, amount_pie, trend_fig

# # ------------------------
# # Callback for General Analysis Summary Boxes
# # ------------------------
# @app.callback(
#     Output('general-summary-boxes', 'children'),
#     [Input('date-picker-range', 'start_date'),
#      Input('date-picker-range', 'end_date')]
# )
# def update_general_summary_boxes(start_date, end_date):
#     df_filtered = df[(df['Transaction_Date'] >= start_date) & (df['Transaction_Date'] <= end_date)]
#     income = df_filtered['Credit'].sum()
#     expenditure = df_filtered['Debit'].sum()
#     balance = income - expenditure
#     total_tx = len(df_filtered)
#     boxes = [
#          summary_box("Income", f"₹{income:,.2f}", bg_color='#d4edda', text_color='#155724'),
#          summary_box("Expenditure", f"₹{expenditure:,.2f}", bg_color='#f8d7da', text_color='#721c24'),
#          summary_box("Balance", f"₹{balance:,.2f}", bg_color='#d1ecf1', text_color='#0c5460'),
#          summary_box("Total Transactions", total_tx, bg_color='#fff3cd', text_color='#856404')
#     ]
#     return boxes

# # ------------------------
# # Callbacks for Breakdown Analysis Tab
# # ------------------------
# # Update Month dropdown based on selected Year
# @app.callback(
#     Output('month-dropdown', 'options'),
#     Input('year-dropdown', 'value')
# )
# def update_month_dropdown(selected_year):
#     if not selected_year:
#         return []
#     months = df[df['Year'] == selected_year][['Month_Num', 'Month']].drop_duplicates()
#     return [{'label': row['Month'], 'value': row['Month_Num']} for _, row in months.iterrows()]

# # Update Week dropdown based on selected Year and Month
# @app.callback(
#     Output('week-dropdown', 'options'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value')]
# )
# def update_week_dropdown(selected_year, selected_month):
#     if not selected_year or not selected_month:
#         return []
#     weeks = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)][['Week_Num']].drop_duplicates()
#     return [{'label': f'Week {w}', 'value': w} for w in sorted(weeks['Week_Num'])]

# # Yearly Summary: Group by Month within the selected Year
# @app.callback(
#     Output('yearly-summary', 'figure'),
#     Input('year-dropdown', 'value')
# )
# def update_yearly_summary(selected_year):
#     if not selected_year:
#         return {}
#     df_filtered = df[df['Year'] == selected_year]
#     df_summary = generate_summary(df_filtered, 'Month')
#     fig = create_bar_chart(df_summary, 'Month', f'Year {selected_year} Summary')
#     return fig

# # Monthly Summary: Group by Week within the selected Year and Month
# @app.callback(
#     Output('monthly-summary', 'figure'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value')]
# )
# def update_monthly_summary(selected_year, selected_month):
#     if not selected_year or not selected_month:
#         return {}
#     df_filtered = df[(df['Year'] == selected_year) & (df['Month_Num'] == selected_month)]
#     df_summary = generate_summary(df_filtered, 'Week_Num')
#     df_summary['Week_Num'] = df_summary['Week_Num'].apply(lambda x: f'Week {x}')
#     fig = create_bar_chart(df_summary, 'Week_Num', f'{df_filtered["Month"].iloc[0]} Summary')
#     return fig

# # Weekly Summary: Group by Day within the selected Year, Month, and Week
# @app.callback(
#     Output('weekly-summary', 'figure'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value'),
#      Input('week-dropdown', 'value')]
# )
# def update_weekly_summary(selected_year, selected_month, selected_week):
#     if not selected_year or not selected_month or not selected_week:
#         return {}
#     df_filtered = df[
#         (df['Year'] == selected_year) &
#         (df['Month_Num'] == selected_month) &
#         (df['Week_Num'] == selected_week)
#     ]
#     df_summary = generate_summary(df_filtered, 'Day')
#     fig = create_bar_chart(df_summary, 'Day', f'Week {selected_week} Summary')
#     return fig

# # Daily Summary: Also group by Day (can be used to show detailed daily info for the selected week)
# @app.callback(
#     Output('daily-summary', 'figure'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value'),
#      Input('week-dropdown', 'value')]
# )
# def update_daily_summary(selected_year, selected_month, selected_week):
#     if not selected_year or not selected_month or not selected_week:
#         return {}
#     df_filtered = df[
#         (df['Year'] == selected_year) &
#         (df['Month_Num'] == selected_month) &
#         (df['Week_Num'] == selected_week)
#     ]
#     df_summary = generate_summary(df_filtered, 'Day')
#     fig = create_bar_chart(df_summary, 'Day', f'Daily Summary for Week {selected_week}')
#     return fig

# # Callback for Breakdown Analysis Summary Boxes
# @app.callback(
#     Output('breakdown-summary-boxes', 'children'),
#     [Input('year-dropdown', 'value'),
#      Input('month-dropdown', 'value'),
#      Input('week-dropdown', 'value')]
# )
# def update_breakdown_summary_boxes(selected_year, selected_month, selected_week):
#     df_filtered = df.copy()
#     if selected_year:
#         df_filtered = df_filtered[df_filtered['Year'] == selected_year]
#     if selected_month:
#         df_filtered = df_filtered[df_filtered['Month_Num'] == selected_month]
#     if selected_week:
#         df_filtered = df_filtered[df_filtered['Week_Num'] == selected_week]
#     income = df_filtered['Credit'].sum()
#     expenditure = df_filtered['Debit'].sum()
#     balance = income - expenditure
#     total_tx = len(df_filtered)
#     boxes = [
#          summary_box("Income", f"₹{income:,.2f}", bg_color='#d4edda', text_color='#155724'),
#          summary_box("Expenditure", f"₹{expenditure:,.2f}", bg_color='#f8d7da', text_color='#721c24'),
#          summary_box("Balance", f"₹{balance:,.2f}", bg_color='#d1ecf1', text_color='#0c5460'),
#          summary_box("Total Transactions", total_tx, bg_color='#fff3cd', text_color='#856404')
#     ]
#     return boxes

# # ------------------------
# # Run the Dash App
# # ------------------------
# if __name__ == '__main__':
#     app.run_server(debug=True, port=8060)
